# 🧠 GPT 환각 평가 파이프라인 (형법 조문 + 판례 요약)
- 질문 + GPT 응답
- 형법 조문 기반 유사도 + NLI 평가
- 판례 요약 기반 유사도 + NLI 평가

In [ ]:
!pip install -q transformers sentence-transformers pandas

In [ ]:
!git clone https://github.com/kimyeonkyeong/gpt_hallucination_risk_management.git
%cd gpt_hallucination_risk_management


fatal: destination path 'gpt_hallucination_risk_management' already exists and is not an empty directory.
/content/gpt_hallucination_risk_management


In [ ]:
# ✅ 기본 모듈 로드
import pandas as pd
import json
from check_similarity import find_most_similar_laws, find_most_similar_cases
from check_entailment import check_entailment

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ✅ 데이터 로드
df = pd.read_csv("law_articles.csv")
with open("example_qa.json", "r", encoding="utf-8") as f:
    qa_data = json.load(f)
with open("case_laws.json", "r", encoding="utf-8") as f:
    case_data = json.load(f)
case_df = pd.DataFrame(case_data)

In [ ]:
# ✅ 평가 실행 (처음 1개 예시)
for idx, item in enumerate(qa_data[:1]):
    question = item["question"]
    gpt_answer = item["gpt_answer"]

    print("="*100)
    print(f"📌 질문 {idx+1}: {question}")
    print(f"📌 GPT 응답: {gpt_answer}\n")

    # 🔍 형법 조문 평가
    filtered = df  # 전체 조문 사용 (필터링은 선택사항)
    top_laws = find_most_similar_laws(question, gpt_answer, filtered)
    print("📚 형법 조문 기반 평가")
    for i, law in enumerate(top_laws):
        print(f"▶ 유사 조문 {i+1}: {law['조문내용']}")
        label, probs = check_entailment(law['조문내용'], gpt_answer)
        print(f"   ⮕ NLI 결과: {label} / 신뢰도: {probs}\n")

    # 🔍 판례 요약 평가
    top_cases = find_most_similar_cases(question, gpt_answer, case_df)
    print("📚 판례 요약 기반 평가")
    for i, case in enumerate(top_cases):
        print(f"▶ 유사 판례 {i+1}: {case['summary']}")
        label, probs = check_entailment(case['summary'], gpt_answer)
        print(f"   ⮕ NLI 결과: {label} / 신뢰도: {probs}\n")


📌 질문 1: 살인죄의 형벌은 무엇입니까?
📌 GPT 응답: 살인죄는 사형, 무기 또는 5년 이상의 징역에 처합니다.
▶ 유사 조문 1: ① 사람의 촉탁이나 승낙을 받아 그를 살해한 자는 1년 이상 10년 이하의 징역에 처한다. ② 사람을 교사하거나 방조하여 자살하게 한 자도 제1항의 형에 처한다. [전문개정 2020. 12. 8.]
   ⮕ NLI 결과: neutral / 신뢰도: [0.3776, 0.6224]
▶ 유사 조문 2: ① 경합범을 동시에 판결할 때에는 다음 각 호의 구분에 따라 처벌한다. 1. 가장 무거운 죄에 대하여 정한 형이 사형, 무기징역, 무기금고인 경우에는 가장 무거운 죄에 대하여 정한 형으로 처벌한다. 2. 각 죄에 대하여 정한 형이 사형, 무기징역, 무기금고 외의 같은 종류의 형인 경우에는 가장 무거운 죄에 대하여 정한 형의 장기 또는 다액(多額)에 그 2분의 1까지 가중하되 각 죄에 대하여 정한 형의 장기 또는 다액을 합산한 형기 또는 액수를 초과할 수 없다. 다만, 과료와 과료, 몰수와 몰수는 병과(倂科)할 수 있다. 3. 각 죄에 대하여 정한 형이 무기징역, 무기금고 외의 다른 종류의 형인 경우에는 병과한다. ② 제1항 각 호의 경우에 징역과 금고는 같은 종류의 형으로 보아 징역형으로 처벌한다. [전문개정 2020. 12. 8.]
   ⮕ NLI 결과: neutral / 신뢰도: [0.3614, 0.6386]
▶ 유사 조문 3: 강도가 사람을 살해한 때에는 사형 또는 무기징역에 처한다. 사망에 이르게 한 때에는 무기 또는 10년 이상의 징역에 처한다. [전문개정 1995. 12. 29.]
   ⮕ NLI 결과: neutral / 신뢰도: [0.376, 0.624]
